In [36]:
import pymongo
import requests
import json
import time
import re
from config import api_key

In [37]:
headers = {'Authorization': 'Bearer %s' % api_key}

In [38]:
# connecting to database
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.restaurant_db

In [43]:
# querying all records
restaurants = db.restaurant_atomic.find()

In [40]:
# convert phone number to format compatible with Yelp's API
def convert_phone(phone):
    phone = re.sub("[(]", "+1",phone)
    phone = re.sub("[)-]","",phone)
    phone = phone.replace(" ","")
    return phone

def get_match_data(id_number):
    # retrieve business information
    time.sleep(1)
    details_response = requests.get(f"https://api.yelp.com/v3/businesses/{id_number}", headers=headers).json()
    try:
        categories = details_response["categories"]
    except:
        categories = "N/A"
    try:
        coordinates = details_response["coordinates"]
    except:
        coordinates = "N/A"
    try:
        photos = details_response["photos"]
    except:
        photos = "N/A"
    try:
        location = details_response["location"]
    except KeyError:
        location = "N/A"
    try:
        price = details_response["price"]
    except KeyError:
        price = "N/A"
    try:
        hours = details_response["hours"]
    except KeyError:
        hours = "N/A"
    try:
        alias = details_response["alias"]
    except KeyError:
        alias = "N/A"
    try:
        image_url = details_response["image_url"]
    except KeyError:
        image_url = "N/A"
    try:
        is_closed = details_response["is_closed"]
    except KeyError:
        is_closed = "N/A"
    try:
        rating = details_response["rating"]
    except KeyError:
        rating = "N/A"
    try:
        review_count = details_response["review_count"]
    except KeyError:
        review_count = "N/A"
    try:
        transactions = details_response["transactions"]
    except KeyError:
        transactions = "N/A"

    # retrieve reviews of business
    reviews_response = requests.get(f"https://api.yelp.com/v3/businesses/{id_number}/reviews", headers=headers)
    time.sleep(1)
    try:
        reviews = reviews_response.json()["reviews"]
    except KeyError:
        reviews = "N/A"

    # create object to insert into database
    restaurant_dict = {
        "restaurant_name": name,
        "alias": alias,
        "is_closed": is_closed,
        "categories": categories,
        "price": price,
        "hours": hours,
        "transactions": transactions,
        "image_url": image_url,
        "pictures": photos,
        "location": location,
        "coordinates": coordinates,
        "rating": rating,
        "review_count": review_count,
        "reviews": reviews
    }

    # insert object into database
    db.yelp.insert_one(restaurant_dict)

def delete_record(name, address1, city, state):
    db.restaurant_atomic.delete_one({"name":name, "street_address":address1})
    print(f"Deleted record for '{name}' in {city}, {state}")  

In [41]:
for restaurant in restaurants[:50]:
    name = restaurant["name"]
    address1 = restaurant["street_address"]
    city = restaurant["city"]
    state = "CO"
    country = "US"
    phone = restaurant["phone"]
    # request Yelp's Business Match API
    match_base_url = "https://api.yelp.com/v3/businesses/matches?"
    match_response = requests.get(f"{match_base_url}name={name}&address1={address1}&city={city}&state={state}&country={country}", headers=headers)
    time.sleep(1)
    # if a match is found, extract information
    if match_response.json()["businesses"]:
        # get Yelp's unique business ID number
        id_number = match_response.json()["businesses"][0]["id"]
        # use unique ID to extract information
        get_match_data(id_number)

    # if no match is found, search by phone number
    elif not match_response.json()["businesses"]:
        # if restaurant_db phone number is in correct format, continue
        if len(restaurant["phone"]) == 14:
            # convert phone number to format compatible with Yelp's Phone Search API
            phone = convert_phone(phone)
            # request Yelp's Phone Search API
            phone_base_url = "https://api.yelp.com/v3/businesses/search/phone?"
            phone_response = requests.get(f"{phone_base_url}phone={phone}", headers=headers)
            time.sleep(1)
            if phone_response.json()["businesses"]:
                # get Yelp's unique business ID number
                id_number = phone_response.json()["businesses"][0]["id"]
                # use unique ID to extract information
                get_match_data(id_number)    
            else:
#                 delete_record(name, address1, city, state)
                print("Delete record")
        else:
#             delete_record(name, address1, city, state)
            print("Delete record")
    else:
#         delete_record(name, address1, city, state)
        print("Delete record")

Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
Delete record
